In [7]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

In [8]:
IMG_SHAPE = 224 # match image dimension to mobile net input

In [9]:
#data sugmentation
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
train_set = train_datagen.flow_from_directory('Soil_Dataset/Train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                class_mode="categorical")

Found 715 images belonging to 4 classes.


In [10]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
testing_set = test_datagen.flow_from_directory('Soil_Dataset/Test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode="categorical")

Found 188 images belonging to 4 classes.


In [11]:
# getting MobileNet
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile_net = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE, 3))

In [12]:
mobile_net.trainable = False

In [13]:
model = tf.keras.models.Sequential([
    mobile_net,
    tf.keras.layers.Dense(4, activation='softmax') #[0, 1] or [1, 0]
    ])

In [14]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit_generator(
    train_set,
    epochs=20,
    validation_data=testing_set
    )

C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
23/23 [==============================] - 64s 3s/step - loss: 1.2271 - accuracy: 0.4775 - val_loss: 0.7399 - val_accuracy: 0.6702
Epoch 2/20
23/23 [==============================] - 51s 2s/step - loss: 0.4283 - accuracy: 0.8644 - val_loss: 0.5650 - val_accuracy: 0.7766
Epoch 3/20
23/23 [==============================] - 58s 3s/step - loss: 0.2993 - accuracy: 0.9196 - val_loss: 0.4170 - val_accuracy: 0.8191
Epoch 4/20
23/23 [==============================] - 58s 3s/step - loss: 0.2578 - accuracy: 0.9158 - val_loss: 0.3917 - val_accuracy: 0.8191
Epoch 5/20
23/23 [==============================] - 57s 2s/step - loss: 0.2199 - accuracy: 0.9349 - val_loss: 0.4278 - val_accuracy: 0.8511
Epoch 6/20
23/23 [==============================] - 54s 2s/step - loss: 0.1839 - accuracy: 0.9331 - val_loss: 0.3754 - val_accuracy: 0.8511
Epoch 7/20
23/23 [==============================] - 57s 2s/step - loss: 0.1700 - accuracy: 0.9382 - val_loss: 0.3822 - val_accuracy: 0.8511
Epoch 8/20
23/23 [==

In [16]:
#save model
from tensorflow.keras.models import load_model
model.save('soil_mobilenet.h5')

In [17]:
from tensorflow.keras.models import load_model
model = load_model(('soil_mobilenet.h5'),custom_objects={'KerasLayer':hub.KerasLayer})

In [31]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('4.png', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
result = np.argmax(result)
soil_type = result

In [32]:
result

3

In [33]:
if soil_type==0:
  print('This Is Alluvial_Soil')
elif soil_type ==1:
  print('This is Black Soil')
elif soil_type == 2:
  print('This is Clay Soil')
else:
  print('This is Red Soil')

This is Red Soil
